In [ ]:
from copy import deepcopy
import pandas as pd
import pickle
import numpy as np
import os
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow_gnn as tfgnn
from keras.engine.input_layer import InputLayer
from keras.models import load_model
from keras.layers import Activation
from keras import backend as K
import keras
from tensorflow.keras import datasets, layers, models

tf.keras.backend.set_floatx('float32')

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
data_path = "../data/mcb-7/ibmq_belem/Pauli_Stochastic/"
x_path = data_path + "circuits/all_circuits_11.npz"
y_path = data_path + "indices/all_circuit_indices_11.npz"

In [ ]:
x_data = np.load(x_path)
x_train = x_data["train"]
x_val = x_data["validate"]
x_test = x_data["test"]

y_data = np.load(y_path)
y_train = y_data["train"]
y_val = y_data["validate"]
y_test = y_data["test"]

In [ ]:
# 1st dimension: Index for training samples
# 2nd dimension: Index for Qubits (5 for ibm Belem)
# 3rd dimension: Index for moments of quantum circuit (zero padding to maximum)
# 4th dimension: One-hot encoding of different quantum gates (+ additional state info)
print(x_train.shape)
print(y_train.shape)

In [ ]:
def create_graph_tensor(numpy_encoding, qubit_adjacency=[[0,1], [1,2], [1,3], [3,4]]):
    """ Takes the 3-tensor numpy encoding of one quantum circuit and the architecture of a quantum computer
    and turns the numpy encoding into a graph tensor.
    """

    nmoments = np.where(np.sum((numpy_encoding != 0), axis=(0,2)) == 0)[0]
    if len(nmoments) == 0:
        nmoments = numpy_encoding.shape[1]
    else:
        nmoments = nmoments[0]

    nqubits = numpy_encoding.shape[0]
    gate_encoding_size = numpy_encoding.shape[2]
    qubit_adjacency = np.array(qubit_adjacency, dtype="int32")
    edge_source_idx = qubit_adjacency

    for i in range(1,nmoments):
        edge_source_idx = np.concatenate((edge_source_idx, qubit_adjacency+(i*nqubits)), axis=0, dtype="int32")
    
    if nmoments > 1:
        qubit_idx = np.arange(nqubits, dtype="int32").reshape(nqubits,-1)
        qubit_idx = np.concatenate((qubit_idx, qubit_idx+nqubits), axis=1)
        moment_adjacency = qubit_idx

        for i in range(2,nmoments):
            moment_adjacency = np.concatenate((moment_adjacency, qubit_idx+(i-1)*nqubits), axis=0, dtype="int32")

        edge_source_idx = np.concatenate((edge_source_idx, moment_adjacency), axis=0, dtype="int32")


    graph_tensor = tfgnn.GraphTensor.from_pieces(
        node_sets = {
            "Qubits": tfgnn.NodeSet.from_fields(
                sizes = [nqubits*nmoments],
                features ={
                    "Gate": np.transpose(numpy_encoding[:,0:nmoments,:], axes=(1,0,2)).reshape(nqubits*nmoments, gate_encoding_size)})},
        edge_sets = {
            "Connectivity": tfgnn.EdgeSet.from_fields(
                sizes = [len(qubit_adjacency)*nmoments+(nmoments-1)*nqubits],
                adjacency = tfgnn.Adjacency.from_indices(
                    source = ("Qubits", edge_source_idx[:,0]),
                    target = ("Qubits", edge_source_idx[:,1])))
        }
    )

    return graph_tensor

In [ ]:
def train_data_generator():
    for (circuit, pst) in zip(x_train, y_train):
        graph = create_graph_tensor(circuit)
        yield graph, pst

In [ ]:
BUFFER_SIZE = 200
BATCH_SIZE = 1
DIM_GATE_VECTOR = 10
GLOBAL_FEATURES = 6

In [ ]:
train_ds = tfgnn.dataset_from_generator(train_data_generator).batch(BATCH_SIZE)
train_dataset = train_ds.shuffle(BUFFER_SIZE)